<a href="https://colab.research.google.com/github/offthetab/NN_homework/blob/main/NLP_sentiment_analysis_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading IMDB dataset from kaggle for sentiment analysis

In [ ]:
%%capture
!pip install kaggle
!pip install pandas==2.2

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
%%capture
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download 'lakshmi25npathi/imdb-dataset-of-50k-movie-reviews'

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:00<00:00, 79.7MB/s]
100% 25.7M/25.7M [00:00<00:00, 79.3MB/s]


In [ ]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


## Importing modules

In [ ]:
import pandas as pd
import numpy as np
import torch as th
from torch.utils.data import DataLoader, Dataset

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


from gensim.models import Word2Vec, FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.similarities import Similarity

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import re
import string

In [ ]:
data = pd.read_csv('IMDB Dataset.csv')
pd.set_option('display.max_colwidth', None)
data.head(5)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

## Preprocess data

In [ ]:
# cleaning reviews and lowering
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text) # Remove HTML tags
    text = re.sub(r'http\S+', '', text) # Remove HTTP/HTTPS links
    text = re.sub(r'<[^>]*>', '', text) # Remove everything inside angle brackets
    text = re.sub(r'\b\d+\b', '', text) # Remove standalone numbers
    text = re.sub(r'\b\w*\d\w*\b', '', text) # Remove words that contain numbers
    return text.lower().strip()

cleared_reviews = data['review'].apply(clean_text)

In [ ]:
# removing punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

no_punctuation_reviews = cleared_reviews.apply(remove_punctuation)
no_punctuation_reviews.head(2)

,review
0,one of the other reviewers has mentioned that after watching just oz episode youll be hooked they are right as this is exactly what happened with methe first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordit is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awayi would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare forget pretty pictures painted for mainstream audiences forget charm forget romanceoz doesnt mess around the first episode i ever saw struck me as so nasty it was surreal i couldnt say i was ready for it but as i watched more i developed a taste for oz and got accustomed to the high levels of graphic violence not just violence but injustice crooked guards wholl be sold out for a nickel inmates wholl kill on order and get away with it well mannered middle class inmates being turned into prison bitches due to their lack of street skills or prison experience watching oz you may become comfortable with what is uncomfortable viewingthats if you can get in touch with your darker side
1,a wonderful little production the filming technique is very unassuming very oldtimebbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great masters of comedy and his life the realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwells murals decorating every surface are terribly well done


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# removing stop words

stop_words = set(nltk.corpus.stopwords.words('english'))

def stopwords_remove(document):
    words = nltk.tokenize.word_tokenize(document) # list of words in document
    tokenized_list = [word for word in words if word not in stop_words]
    return ' '.join(tokenized_list)

nosw_reviews = no_punctuation_reviews.apply(lambda x: stopwords_remove(x))
nosw_reviews.head(2)

,review
0,one reviewers mentioned watching oz episode youll hooked right exactly happened methe first thing struck oz brutality unflinching scenes violence set right word go trust show faint hearted timid show pulls punches regards drugs sex violence hardcore classic use wordit called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home manyaryans muslims gangstas latinos christians italians irish moreso scuffles death stares dodgy dealings shady agreements never far awayi would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannered middle class inmates turned prison bitches due lack street skills prison experience watching oz may become comfortable uncomfortable viewingthats get touch darker side
1,wonderful little production filming technique unassuming oldtimebbc fashion gives comforting sometimes discomforting sense realism entire piece actors extremely well chosen michael sheen got polari voices pat truly see seamless editing guided references williams diary entries well worth watching terrificly written performed piece masterful production one great masters comedy life realism really comes home little things fantasy guard rather use traditional dream techniques remains solid disappears plays knowledge senses particularly scenes concerning orton halliwell sets particularly flat halliwells murals decorating every surface terribly well done


In [ ]:
# corpus to train word2vec models
corpus = [doc.split(' ') for doc in nosw_reviews.tolist()]

In [ ]:
# encoding target
bin_sentiment = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

## Training Word2vec CBOW model



In [ ]:
# training Word2vec model on X_train and saving model
cbow_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=5, workers=4, sg=0) # if sg = 0 => CBOW; if sg = 1 => Skip-gram

cbow_model.save("CBOW.model")

In [ ]:
# most similar words
sims = cbow_model.wv.most_similar('diamond', topn=20)
sims

[('microfilm', 0.7870280146598816),
 ('owner', 0.7791194319725037),
 ('robber', 0.7741398215293884),
 ('pin', 0.7678897380828857),
 ('dibiase', 0.7606232166290283),
 ('banks', 0.7595071792602539),
 ('logan', 0.7585260272026062),
 ('lookalike', 0.7575277090072632),
 ('mobster', 0.7518430948257446),
 ('chauffeur', 0.7517816424369812),
 ('proprietor', 0.7506452202796936),
 ('supplies', 0.7489153146743774),
 ('abu', 0.7473299503326416),
 ('employee', 0.747032880783081),
 ('malden', 0.7458301782608032),
 ('frankie', 0.7454890608787537),
 ('warden', 0.7453808784484863),
 ('lin', 0.7449530959129333),
 ('chin', 0.7446602582931519),
 ('trusty', 0.7441987991333008)]

In [ ]:
# most similar words
sims = cbow_model.wv.most_similar('good', topn=10)
sims

[('decent', 0.7723395228385925),
 ('great', 0.7302871942520142),
 ('bad', 0.6465182900428772),
 ('nice', 0.6401995420455933),
 ('ok', 0.6162018179893494),
 ('okay', 0.6123636364936829),
 ('alright', 0.6060921549797058),
 ('fine', 0.5977991819381714),
 ('cool', 0.5916581153869629),
 ('solid', 0.5905265212059021)]

In [ ]:
# number of docs in corpus
cbow_model.corpus_count

50000

In [ ]:
# Get the vocabulary as a set of words
vocabulary = set(cbow_model.wv.index_to_key)

In [ ]:
# number of tokens
len(vocabulary)

43641

### To represent documents (sentences) as a vector we will average all word embeddings

In [ ]:
def doc_mean_embedding(doc):
    doc = doc.split(' ')
    return np.mean([cbow_model.wv[word] for word in doc if word in vocabulary], axis=0)

In [ ]:
# example of first document mean embedding
doc_mean_embedding(nosw_reviews[0])

array([-0.12782374,  0.36052907, -0.5085584 , -0.18030353,  0.17822514,
       -0.28750607,  0.44882488,  0.1080685 , -0.55562484, -0.19082914,
        0.03333442, -0.17671652, -0.23748043,  0.1671974 ,  0.04588289,
       -0.11657263, -0.05613854, -0.36907086,  0.36776268, -0.31888688,
       -0.13293847, -0.05520891,  0.19197068, -0.0435289 ,  0.15715635,
       -0.08249204, -0.33581057, -0.08255807,  0.02865516, -0.13731715,
        0.5706256 ,  0.09217656, -0.26428026, -0.1391153 , -0.31340343,
        0.1266245 , -0.01932101,  0.04233512, -0.1989525 , -0.05047514,
       -0.10594375,  0.11242185,  0.08293273, -0.29464385,  0.05114777,
       -0.24343391, -0.24395013, -0.23977618, -0.27061346,  0.06306761,
        0.28853106,  0.01046828,  0.3126878 ,  0.13505165,  0.18845724,
        0.17501126,  0.4199241 , -0.07916828, -0.22492197,  0.27410918,
        0.20267884, -0.18858424,  0.07208639,  0.10279194, -0.2545472 ,
        0.04932604, -0.08071173,  0.4056108 , -0.02544387,  0.08

## Training Support Vector Classifier on CBOW embeddings

for mean document embedding we will use [model.wv.get_mean_vector()](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.get_mean_vector) instead of doc_mean_embedding() written above.

In [ ]:
reviews = []
for doc in nosw_reviews.to_numpy():
    reviews.append(cbow_model.wv.get_mean_vector(doc.split(' '), pre_normalize=False))

reviews = np.stack(np.array(reviews))
sentiments = bin_sentiment.to_numpy()
reviews.shape

(50000, 100)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    reviews,
    sentiments,
    test_size=0.2,
    shuffle=True,
    random_state=42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40000, 100), (10000, 100), (40000,), (10000,))

In [ ]:
SVM_classifier = SVC(kernel='rbf', random_state=0).fit(X_train, y_train)
y_preds = SVM_classifier.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_preds))
print(classification_report(y_preds, y_test))

0.8684
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      4871
           1       0.88      0.86      0.87      5129

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## Training Word2vec skip-gram model


In [ ]:
# training Word2vec skip-gram model on X_train and saving model
sg_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=5, workers=4, sg=1) # if sg = 0 => CBOW; if sg = 1 => Skip-gram

sg_model.save("skip_gram.model")

In [ ]:
sg_model.wv.most_similar('bad', topn=10)

[('awful', 0.7936199903488159),
 ('awfulthe', 0.7808029055595398),
 ('terrible', 0.7792365550994873),
 ('horrible', 0.7730497121810913),
 ('badi', 0.7673512697219849),
 ('badbut', 0.7649745345115662),
 ('hilariousthe', 0.7597746253013611),
 ('badthe', 0.7543224692344666),
 ('good', 0.7502711415290833),
 ('terriblethe', 0.7361679673194885)]

In [ ]:
sg_model.wv.most_similar('good', topn=10)

[('decent', 0.8311371803283691),
 ('goodand', 0.8249462842941284),
 ('great', 0.7854470014572144),
 ('moviemost', 0.7697486281394958),
 ('greatthe', 0.7688313722610474),
 ('actorthe', 0.7572321891784668),
 ('goodoverall', 0.7567346692085266),
 ('workable', 0.7561923265457153),
 ('unexceptional', 0.7518524527549744),
 ('bad', 0.7502711415290833)]

In [ ]:
# number of docs in corpus
sg_model.corpus_count

50000

In [ ]:
# number of tokens
len(sg_model.wv.index_to_key)

43641

## Training Support Vector Classifier on skip-gram embeddings

In [ ]:
reviews_sg = []
for doc in nosw_reviews.to_numpy():
    reviews_sg.append(sg_model.wv.get_mean_vector(doc.split(' '), pre_normalize=False))

reviews_sg = np.stack(np.array(reviews_sg))
sentiments_sg = bin_sentiment.to_numpy()
reviews_sg.shape, sentiments_sg.shape

((50000, 100), (50000,))

In [ ]:
X_train_sg, X_test_sg, y_train_sg, y_test_sg = train_test_split(
    reviews_sg,
    sentiments_sg,
    test_size=0.2,
    shuffle=True,
    random_state=42
)

X_train_sg.shape, X_test_sg.shape, y_train_sg.shape, y_test_sg.shape

((40000, 100), (10000, 100), (40000,), (10000,))

In [ ]:
SVM_classifier_sg = SVC(kernel='rbf', random_state=0).fit(X_train_sg, y_train_sg)
y_preds_sg = SVM_classifier_sg.predict(X_test_sg)

In [ ]:
print(accuracy_score(y_test_sg, y_preds_sg))
print(classification_report(y_preds_sg, y_test_sg))

0.8795
              precision    recall  f1-score   support

           0       0.87      0.88      0.88      4906
           1       0.89      0.88      0.88      5094

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
log_reg = LogisticRegression(random_state=0, max_iter=10000).fit(X_train_sg, y_train_sg)
y_preds_sgLog = log_reg.predict(X_test_sg)

In [ ]:
print(accuracy_score(y_test_sg, y_preds_sgLog))
print(classification_report(y_preds_sgLog, y_test_sg))

0.8752
              precision    recall  f1-score   support

           0       0.87      0.88      0.87      4917
           1       0.88      0.87      0.88      5083

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



## Training Doc2vec model

In [ ]:
# getting tagged documents for doc2vec

# tags generator
def tags_generator(corpus):
    for idx, doc in enumerate(corpus):
        yield TaggedDocument(doc, [idx])

In [ ]:
# tagged documents
documents = list(tags_generator(corpus))
documents[:2]

[TaggedDocument(words=['one', 'reviewers', 'mentioned', 'watching', 'oz', 'episode', 'youll', 'hooked', 'right', 'exactly', 'happened', 'methe', 'first', 'thing', 'struck', 'oz', 'brutality', 'unflinching', 'scenes', 'violence', 'set', 'right', 'word', 'go', 'trust', 'show', 'faint', 'hearted', 'timid', 'show', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence', 'hardcore', 'classic', 'use', 'wordit', 'called', 'oz', 'nickname', 'given', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focuses', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cells', 'glass', 'fronts', 'face', 'inwards', 'privacy', 'high', 'agenda', 'em', 'city', 'home', 'manyaryans', 'muslims', 'gangstas', 'latinos', 'christians', 'italians', 'irish', 'moreso', 'scuffles', 'death', 'stares', 'dodgy', 'dealings', 'shady', 'agreements', 'never', 'far', 'awayi', 'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'goes', 'shows', 'wouldnt', 'dare', 'forget', 'pretty', 'pictures', '

In [ ]:
doc2vec = Doc2Vec(documents, vector_size=100, window=5, min_count=5, workers=2, dm=1) # dm=1, ‘distributed memory’ (PV-DM); dm=0 distributed bag of words (PV-DBOW)
doc2vec.save('doc2vec.model')

In [ ]:
doc2vec.wv.most_similar('bad', topn=10)

[('terrible', 0.7414311170578003),
 ('awful', 0.7281287908554077),
 ('horrible', 0.7029908895492554),
 ('good', 0.6779029965400696),
 ('atrocious', 0.6092491149902344),
 ('lousy', 0.5923641920089722),
 ('badthe', 0.5807062387466431),
 ('cheesy', 0.570451021194458),
 ('decent', 0.5686554908752441),
 ('lame', 0.5675964951515198)]

In [ ]:
doc2vec.wv.most_similar('good', topn=10)

[('decent', 0.8064809441566467),
 ('great', 0.7746395468711853),
 ('bad', 0.6779030561447144),
 ('nice', 0.6666819453239441),
 ('excellent', 0.6398542523384094),
 ('ok', 0.6297956705093384),
 ('fine', 0.6163005232810974),
 ('cool', 0.5987763404846191),
 ('alright', 0.5853174328804016),
 ('okay', 0.5837872624397278)]

In [ ]:
# top 5 most similar documents to the sentence below
pos = doc2vec.infer_vector(['driver', 'has', 'died'])

sims = doc2vec.dv.most_similar(positive=pos, topn = 5)
ids = [id_sim[0] for id_sim in sims]
cos_sim = [id_sim[1] for id_sim in sims]

df = no_punctuation_reviews[ids].reset_index()
df['cosine_similarity'] = np.array(cos_sim)
df = df.set_index('index')
df

,review,cosine_similarity
index,,
28519,the beginning of this movie was good it started to get really dumb after he told the people he wanted to kill himself i think if i came from a little town like that i would be offended after seeing this movie they made a lot of these people look dumb and crazy how could these people have so little to do that they follow him around all day a lot of times these people were telling him ways to kill himself like they were urging him to do it how can so many people have little respect for other people i also think they could have made a much better ending for this movie there were some good parts to this movie also some people might like it but i wouldnt recommend this to anyone,0.677292
29482,i cant stand it when people go see a movie when they know they wont like it my mom likes violent movies so why did she see it she rated it just to bring down the rating so i know thats why it didnt have a higher rating i give it a,0.676632
33598,a friend gave me this movie because she liked it i decided i would finally watch it it was sooooooo long i kept waiting for the suspense to happen but it never did i kept waiting for something to happen after the opening scenes and it never did i stopped the movie and came back later i actually forced myself to watch the rest of it hoping it would get better it got worse i kept asking myself who are these people do they have feelings are they just robots im glad i didnt pay to see it or pay to rent it the end would have been better if dutch died from the gunshot wound at least we would have gotten some emotion from the audience or maybe not,0.675922
31549,i just saw this film i first saw it when i was and could just about remember the end so i watched it like minutes ago and i may seem like a baby as i am haha i started to cry at the ending i forgotten how sad it was i think i was mainly sad for annemarie because she said i love you charlie and also ill miss you charlie just made me really cry haha it has to be one of me favourite movies of all time it is just a film well worth watching watch it haha thats all i can say xdbut i love this film its a true classicxx maverick xx,0.672652
35651,although there were some amusing moments i thought the movie was pretty lame the longer it ran the worse it got once the action entered monument valley i found myself watching the magnificent outcroppings more than the increasingly silly and unconvincing interaction of the charactersthe character of the daughter was particularly incoherent first shes in on the deal then discovers the truth and she bails then shes back again then deserts them again then shes back again theres no apparent motivation for any of her decisions there were interesting characters some interesting scenes and many missed possibilities i would have to say the pictures was much less than the sum of its parts apparently the people who liked repo man were inclined to like this one searchers is no match for the searchers,0.669892


In [ ]:
doc2vec.corpus_count

50000

In [ ]:
len(doc2vec.wv.index_to_key)

43641

## Training Support Vector Classifier on doc2vec embeddings

In [ ]:
# getting all document embeddings by their tag
doc_embeddings = doc2vec.dv[[i for i in range(doc2vec.corpus_count)]]
sentiments_target = bin_sentiment.to_numpy() # targets

doc_embeddings.shape, sentiments_target.shape

((50000, 100), (50000,))

In [ ]:
X_train_doc, X_test_doc, y_train_doc, y_test_doc = train_test_split(
    doc_embeddings,
    sentiments_target,
    test_size=0.2,
    shuffle=True,
    random_state=42
)

X_train_doc.shape, X_test_doc.shape, y_train_doc.shape, y_test_doc.shape

((40000, 100), (10000, 100), (40000,), (10000,))

In [ ]:
SVM_classifier_doc = SVC(kernel='rbf', random_state=0).fit(X_train_doc, y_train_doc)
y_preds_doc = SVM_classifier_doc.predict(X_test_doc)

In [ ]:
print(accuracy_score(y_test_doc, y_preds_doc))
print(classification_report(y_preds_doc, y_test_doc))

0.8506
              precision    recall  f1-score   support

           0       0.84      0.86      0.85      4855
           1       0.86      0.84      0.85      5145

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [ ]:
# class IMDbDataset(Dataset):
#     def __init__(self, data):
#         self.data = data
#         self.labels = []
#         self.reviews = []

#         for label, review in data:
#             # self.labels.append(1 if label == 'pos' else 0)  # Convert labels to binary
#             # self.reviews.append(preprocess_text(review))
#             self.labels.append(label)
#             self.reviews.append(review)

#     def __len__(self):
#         return len(self.reviews)

#     def __getitem__(self, idx):
#         return self.reviews[idx], self.labels[idx]

In [ ]:
# # Create datasets
# train_dataset = IMDbDataset(train_data)
# test_dataset = IMDbDataset(test_data)